# DATA PROJECT

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
#from matplotlib_venn import venn2

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject

# import dst data loader
import pydst
Dst = pydst.Dst(lang='en') 


pd.set_option('display.max_rows', None)   # show all rows
pd.set_option('display.max_columns', None)   # show all columns


# Read and clean data

Import your data, either through an API or manually, and load it. 

In [2]:
# Data for industrial robots on industry, year for Denmark
filename = 'ifr_dk.csv'
df_ifr = pd.read_csv(filename, delimiter=';')

# Drop obs that are not relevant (data not valid before 2004)
drop = [str(year) for year in range(1993,2004)]
df_ifr.drop(drop, axis=1, inplace=True)
df_ifr = df_ifr.iloc[1:20, :]

# Renaming columns
col_dict = {}
for i in range(2004, 2016+1): 
    col_dict[str(i)] = f'rob{i}' 
col_dict['DK'] = f'industry' 

df_ifr.rename(columns = col_dict, inplace=True)

df_ifr.head(20)



,industry,rob2004,rob2005,rob2006,rob2007,rob2008,rob2009,rob2010,rob2011,rob2012,rob2013,rob2014,rob2015,rob2016
1,"A-B-Agriculture, forestry, fishing",59,64,75,80,84,90,96,98,109,114,109,79,72
2,C-Mining and quarrying,46,45,43,43,43,43,43,27,30,18,19,9,6
3,10-12-Food and beverages,145,165,241,342,395,452,518,583,646,698,844,948,1138
4,13-15-Textiles,95,102,104,104,106,99,98,116,111,121,144,146,145
5,16-Wood and furniture,262,298,320,335,347,347,350,328,313,283,285,242,206
6,17-18-Paper,14,21,24,29,33,35,38,39,51,55,63,70,63
7,19-22-Plastic and chemical products,295,348,404,487,566,575,615,678,754,848,1011,1141,1334
8,"23-Glass, ceramics, stone, mineral products (n...",0,7,24,42,91,104,114,123,131,159,172,190,206
9,24-Basic metals,70,88,102,121,132,127,110,118,142,142,164,179,185
10,25-Metal products (non-automotive),928,954,1006,1107,1180,1189,1123,1078,976,978,957,926,912


In [31]:
# Inspicerer indhold
nabb69_vars = Dst.get_variables(table_id='NABB69')
nabb69_vars.id.unique()

# Importing data from statistikbanken through an API
variables = {'SOCIO':['EMPM_DC'],'BRANCHE':['*'],'Tid':['2004','2005','2006','2007', '2008', '2009', '2010','2011', '2012', '2013', '2014', '2015','2016']}

# Importerer data
nabb69 = Dst.get_data(table_id = 'NABB69', variables=variables)
nabb69.sort_values(by=['SOCIO', 'TID', 'BRANCHE'], inplace=True)

# Ordner og aggregerer industri-koder: Klar til merge med ifr
# Dictionary for oversættelse af industri-koder
industry = nabb69.BRANCHE
dict_nabb69    = {x[:5]: x[6:] for x in industry}
dict_crosswalk = {'01000':'A-B-Agriculture, forestry, fishing',
                  '02000':'A-B-Agriculture, forestry, fishing',
                  '03000':'A-B-Agriculture, forestry, fishing',
                  '06090':'C-Mining and quarrying',
                  '10120': '10-12-Food and beverages',
                  '13150':'13-15-Textiles',
                  '16000': '16-Wood and furniture',
                  '17000':'17-18-Paper',
                  '18000':'17-18-Paper',
                  '19000':'19-22-Plastic and chemical products',
                  '20000':'19-22-Plastic and chemical products',
                  '21000':'19-22-Plastic and chemical products',
                  '22000':'19-22-Plastic and chemical products',
                  '23000':'23-Glass, ceramics, stone, mineral products (n...',
                  '24000':'24-Basic metals',
                  '25000':'25-Metal products (non-automotive)',
                  '28000':'28-Industrial machinery',
                  '26000':'26-27-Electrical/electronics',
                  '27000':'26-27-Electrical/electronics',
                  '29000':'29-Automotive',
                  '30000':'30-Other vehicles',
                  '31320':'91-All other manufacturing branches',
                  '33000':'91-All other manufacturing branches',
                  '35000':'E-Electricity, gas, water supply',
                  '36000':'E-Electricity, gas, water supply',
                  '37390':'E-Electricity, gas, water supply',
                  '41430':'Construction',
                  '72002':'P-Education/research/development',
                  '85202':'P-Education/research/development'}

# Opdaterer dictionary så alle andre kategorier får '90'.
dict_crosswalk.update({
    key: '90-All other non-manufacturing branches'
    for key in range(0, 100000)
    if str(key) not in dict_crosswalk
})

# Tilføjer ny variabel med talkoder for branche og bruger dictionary til at oversætte industrikoder
nabb69['BRANCHE_K'] = nabb69['BRANCHE'].str[:5]
nabb69['BRANCHE_IFR'] = nabb69['BRANCHE_K'].map(dict_crosswalk)

# Aggregerer på IFR-industrier og tid
nabb69['emp_ifr'] = nabb69.groupby(['BRANCHE_IFR', 'TID'])['INDHOLD'].transform(sum)

des = nabb69.describe()
print(des)
print(nabb69.dtypes)

nabb69.head(10)


               TID       INDHOLD        emp_ifr
count   923.000000  9.230000e+02     377.000000
mean   2010.000000  9.114010e+04   48773.432361
std       3.743686  3.449684e+05   55814.500208
min    2004.000000  0.000000e+00    2068.000000
25%    2007.000000  8.135500e+03   13355.000000
50%    2010.000000  1.729600e+04   30143.000000
75%    2013.000000  4.777950e+04   52811.000000
max    2016.000000  2.946869e+06  225049.000000
SOCIO           object
BRANCHE         object
TID              int64
INDHOLD          int64
BRANCHE_K       object
BRANCHE_IFR     object
emp_ifr        float64
dtype: object


,SOCIO,BRANCHE,TID,INDHOLD,BRANCHE_K,BRANCHE_IFR,emp_ifr
783,Employment (number),01000 Agriculture and horticulture,2004,71446,01000,"A-B-Agriculture, forestry, fishing",79857.0
784,Employment (number),02000 Forestry,2004,4386,02000,"A-B-Agriculture, forestry, fishing",79857.0
785,Employment (number),03000 Fishing,2004,4025,03000,"A-B-Agriculture, forestry, fishing",79857.0
786,Employment (number),06090 Mining and quarrying,2004,3492,06090,C-Mining and quarrying,3492.0
787,Employment (number),"10120 Manufacture of food products, beverages ...",2004,69960,10120,10-12-Food and beverages,69960.0
788,Employment (number),13150 Textiles and leather products,2004,9995,13150,13-15-Textiles,9995.0
789,Employment (number),16000 Manufacture of wood and wood products,2004,12614,16000,16-Wood and furniture,12614.0
790,Employment (number),17000 Manufacture of paper and paper products,2004,7587,17000,17-18-Paper,20198.0
791,Employment (number),18000 Printing etc.,2004,12611,18000,17-18-Paper,20198.0
792,Employment (number),19000 Oil refinery etc.,2004,806,19000,19-22-Plastic and chemical products,46831.0


## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [4]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [5]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

NameError: name 'venn2' is not defined

<Figure size 1500x700 with 0 Axes>

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.